# Wine prediction

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer,SimpleImputer

from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, multilabel_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_wine

In [2]:
x = pd.DataFrame(load_wine().data , columns = load_wine().feature_names)
y = pd.Series(load_wine().target)

In [3]:
y.unique()

array([0, 1, 2])

In [5]:
x.isna().sum()

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64

# Train test split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=10, stratify=y)

### By observing we can say that there is no null value or any special character is present in columns

In [7]:
x.corr()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
alcohol,1.000000,0.094397,0.211545,-0.310235,0.270798,0.289101,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.072343,0.643720
malic_acid,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.335167,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.368710,-0.192011
ash,0.211545,0.164045,1.000000,0.443367,0.286587,0.128980,0.115077,0.186230,0.009652,0.258887,-0.074667,0.003911,0.223626
alcalinity_of_ash,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.321113,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.276769,-0.440597
magnesium,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.214401,0.195784,-0.256294,0.236441,0.199950,0.055398,0.066004,0.393351
total_phenols,0.289101,-0.335167,0.128980,-0.321113,0.214401,1.000000,0.864564,-0.449935,0.612413,-0.055136,0.433681,0.699949,0.498115
flavanoids,0.236815,-0.411007,0.115077,-0.351370,0.195784,0.864564,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.787194,0.494193
nonflavanoid_phenols,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.449935,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.503270,-0.311385
proanthocyanins,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.612413,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.519067,0.330417
color_intensity,0.546364,0.248985,0.258887,0.018732,0.199950,-0.055136,-0.172379,0.139057,-0.025250,1.000000,-0.521813,-0.428815,0.316100


In [8]:
df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


# scaling

In [9]:
std_scaler = StandardScaler()
std_scaler.fit(x)  # Saving mean and std

StandardScaler()

In [10]:
standard_array = std_scaler.transform(x)
x_df_std = pd.DataFrame(standard_array, columns = x.columns)
x_df_std

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.876275,2.974543,0.305159,0.301803,-0.332922,-0.985614,-1.424900,1.274310,-0.930179,1.142811,-1.392758,-1.231206,-0.021952
174,0.493343,1.412609,0.414820,1.052516,0.158572,-0.793334,-1.284344,0.549108,-0.316950,0.969783,-1.129518,-1.485445,0.009893
175,0.332758,1.744744,-0.389355,0.151661,1.422412,-1.129824,-1.344582,0.549108,-0.422075,2.224236,-1.612125,-1.485445,0.280575
176,0.209232,0.227694,0.012732,0.151661,1.422412,-1.033684,-1.354622,1.354888,-0.229346,1.834923,-1.568252,-1.400699,0.296498


In [11]:
from imblearn.over_sampling import SMOTE

In [12]:
smt = SMOTE()
x_samples, y_samples = smt.fit_resample(x_df_std,y)
y_samples.value_counts()

0    71
1    71
2    71
dtype: int64

In [13]:
x_samples

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,-0.046465,0.593928,0.962746,1.325642,0.037556,-0.183298,-1.449467,1.311162,-1.450602,-0.104334,-0.295924,-0.675098,-0.539948
209,-0.327389,0.060883,-0.375453,-0.059277,-0.851120,-1.375677,-1.494807,1.196703,-1.559638,1.849249,-1.557752,-1.400872,-0.736702
210,0.493448,2.255498,-0.260146,0.813919,-0.617590,-1.192312,-1.470820,1.701552,-0.359607,1.001833,-1.243782,-1.441753,-0.337941
211,-0.591520,1.167412,-0.434401,-0.155601,0.057296,-1.389920,-1.311664,-0.156399,-0.932020,1.786203,-1.202463,-1.032322,0.086161


# train test split

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)
x_train

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
126,-0.704864,-0.723829,-0.279695,0.602088,-0.964842,0.712858,1.125176,0.226796,0.313800,-0.483652,-1.173391,0.322481,-1.257535
71,1.061565,-0.741782,1.109334,1.653086,-0.964842,1.049347,0.834026,-1.223610,0.489009,-0.725891,1.766126,0.774463,-1.072834
59,-0.778980,-1.253450,-3.679162,-2.671018,-0.824415,-0.504914,-1.465058,-0.659563,-2.051513,-1.344466,0.406051,-1.118210,-0.722540
94,-1.705429,-0.310904,-0.316249,-0.448909,-0.122282,1.161510,0.231646,-1.545922,-0.422075,-0.782125,0.888658,0.491974,-1.279827
116,-1.458376,-0.777689,-1.376297,0.391889,-0.964842,-0.504914,-0.430972,-0.498407,-0.106700,-1.344466,-0.032683,1.014578,-0.802153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.826864,-0.975175,-1.632171,-0.448909,-0.403135,-0.312635,-0.240219,-0.337251,-1.508367,-0.544212,1.195772,-0.214247,-0.372246
16,1.605081,-0.373741,1.292101,0.151661,1.422412,0.808997,1.115136,-0.256673,0.664217,0.493956,0.493797,0.054117,1.697675
185,1.006576,0.927222,0.651949,-1.471415,1.396483,0.794204,1.007738,-1.472783,0.343717,0.018129,-0.081730,1.129601,0.230523
136,-0.927212,2.139716,0.634140,0.451946,-0.754202,-1.466313,-1.565455,1.354888,-1.385721,-0.522583,-0.910151,-1.895054,-0.085641


In [15]:
y_test.value_counts()

0    32
1    32
2    32
dtype: int64

# Model training

In [16]:
log_reg = LogisticRegression(random_state=10)
log_reg.fit(x_train,y_train)

LogisticRegression(random_state=10)

# Evaluation

In [17]:
## Testing daat evaluation
y_pred_test = log_reg.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred_test)
print('Confusion matrix : \n', cnf_matrix)

multilabel_cnf_matrix = multilabel_confusion_matrix(y_test, y_pred_test)
print('Multilabel Confusion matrix : \n', multilabel_cnf_matrix)

acc_score_Logistic_Test = accuracy_score(y_test, y_pred_test)
print('Accuracy score : \n', acc_score_Logistic_Test)

class_report = classification_report(y_test, y_pred_test)
print('Classification report : \n', class_report)


Confusion matrix : 
 [[31  1  0]
 [ 0 32  0]
 [ 0  0 32]]
Multilabel Confusion matrix : 
 [[[64  0]
  [ 1 31]]

 [[63  1]
  [ 0 32]]

 [[64  0]
  [ 0 32]]]
Accuracy score : 
 0.9895833333333334
Classification report : 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98        32
           1       0.97      1.00      0.98        32
           2       1.00      1.00      1.00        32

    accuracy                           0.99        96
   macro avg       0.99      0.99      0.99        96
weighted avg       0.99      0.99      0.99        96



In [18]:
# Training Data Evalutaion

y_pred_train = log_reg.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score__Logistic_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score__Logistic_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[39  0  0]
 [ 0 39  0]
 [ 0  0 39]]
Accuracy score :
 1.0
Classification report :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        39

    accuracy                           1.00       117
   macro avg       1.00      1.00      1.00       117
weighted avg       1.00      1.00      1.00       117



# BY KNN

# Model training

In [19]:
knn_clf = KNeighborsClassifier(n_neighbors=5)  
knn_clf.fit(x_train,y_train)  # model saves training data

KNeighborsClassifier()

In [20]:
# Testing Data Evalutaion

y_pred = knn_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Test = accuracy_score(y_test,y_pred)
print('Accuracy score :\n', acc_score_KNN_Test)

clf_report = classification_report(y_test,y_pred)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[32  0  0]
 [ 1 30  1]
 [ 0  0 32]]
Accuracy score :
 0.9791666666666666
Classification report :
               precision    recall  f1-score   support

           0       0.97      1.00      0.98        32
           1       1.00      0.94      0.97        32
           2       0.97      1.00      0.98        32

    accuracy                           0.98        96
   macro avg       0.98      0.98      0.98        96
weighted avg       0.98      0.98      0.98        96



In [21]:
# Training Data Evalutaion

y_pred_train = knn_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score_KNN_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[39  0  0]
 [ 3 33  3]
 [ 0  0 39]]
Accuracy score :
 0.9487179487179487
Classification report :
               precision    recall  f1-score   support

           0       0.93      1.00      0.96        39
           1       1.00      0.85      0.92        39
           2       0.93      1.00      0.96        39

    accuracy                           0.95       117
   macro avg       0.95      0.95      0.95       117
weighted avg       0.95      0.95      0.95       117



# Decision Tree classification

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)

In [23]:
dt_clf = DecisionTreeClassifier(random_state=10)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(random_state=10)

# Model Evaluation

In [27]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[29  3  0]
 [ 0 32  0]
 [ 1  2 29]]
********************************************
Accuracy Score : 
 0.9375
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94        32
           1       0.86      1.00      0.93        32
           2       1.00      0.91      0.95        32

    accuracy                           0.94        96
   macro avg       0.94      0.94      0.94        96
weighted avg       0.94      0.94      0.94        96

********************************************


In [28]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[39  0  0]
 [ 0 39  0]
 [ 0  0 39]]
********************************************
Accuracy Score : 
 1.0
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        39

    accuracy                           1.00       117
   macro avg       1.00      1.00      1.00       117
weighted avg       1.00      1.00      1.00       117

********************************************


# Plot DT

In [29]:
plt.figure(figsize = (200,160))
plot_tree(dt_clf, feature_names=x.columns, class_names = ["0", "1",'2'], filled = True)
plt.savefig('DecisionTree_wine.png')

# Hyperparameter Testing

In [30]:
dt_clf = DecisionTreeClassifier()
hyperparameters = {"criterion" : ["gini", "entropy"],
                  "max_depth" : np.arange(3,8),
                  "min_samples_split" : np.arange(2,20),
                  "min_samples_leaf" : np.arange(2,10)}

In [31]:
rscv_dt_model = RandomizedSearchCV(dt_clf, hyperparameters, cv = 5)
rscv_dt_model.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([3, 4, 5, 6, 7]),
                                        'min_samples_leaf': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])})

In [32]:
rscv_dt_model.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       min_samples_split=12)

In [33]:
dt_clf = DecisionTreeClassifier(criterion='entropy',max_depth=3, min_samples_leaf=5,
                       min_samples_split=12)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       min_samples_split=12)

In [34]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[30  2  0]
 [ 2 29  1]
 [ 0  5 27]]
********************************************
Accuracy Score : 
 0.8958333333333334
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.94      0.94      0.94        32
           1       0.81      0.91      0.85        32
           2       0.96      0.84      0.90        32

    accuracy                           0.90        96
   macro avg       0.90      0.90      0.90        96
weighted avg       0.90      0.90      0.90        96

********************************************


In [35]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[39  0  0]
 [ 2 37  0]
 [ 0  2 37]]
********************************************
Accuracy Score : 
 0.9658119658119658
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97        39
           1       0.95      0.95      0.95        39
           2       1.00      0.95      0.97        39

    accuracy                           0.97       117
   macro avg       0.97      0.97      0.97       117
weighted avg       0.97      0.97      0.97       117

********************************************


# Random Forest

In [36]:
rf_clf = RandomForestClassifier(random_state=10)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=10)

In [39]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Test = accuracy_score(y_test,y_pred)
print("Accuracy score \n", acc_score_RF_Test)

clf_report = classification_report(y_test,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[32  0  0]
 [ 0 32  0]
 [ 0  0 32]]
Accuracy score 
 1.0
Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00        32

    accuracy                           1.00        96
   macro avg       1.00      1.00      1.00        96
weighted avg       1.00      1.00      1.00        96



In [40]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Train = accuracy_score(y_train,y_pred)
print("Accuracy score \n", acc_score_RF_Train)

clf_report = classification_report(y_train,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[39  0  0]
 [ 0 39  0]
 [ 0  0 39]]
Accuracy score 
 1.0
Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        39

    accuracy                           1.00       117
   macro avg       1.00      1.00      1.00       117
weighted avg       1.00      1.00      1.00       117



# Hyperparameter Tuning

In [41]:
rf_model = RandomForestClassifier(random_state=10)

hyp = {"n_estimators" : np.arange(10,200),
    "criterion":['gini','entropy'],
    "max_depth":np.arange(5,15),
    "min_samples_split":np.arange(5,20),
    "min_samples_leaf":np.arange(5,15),
    "max_features":['auto']}

In [42]:
rscv_rf_clf =RandomizedSearchCV(rf_model, hyp, cv = 5)
rscv_rf_clf.fit(x_train,y_train)
rscv_rf_clf.best_estimator_

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=12, max_features='auto', min_samples_leaf=14,
                       min_samples_split=14, n_estimators=141, random_state=10)

In [43]:
rf_clf = RandomForestClassifier(max_depth=12, max_features='auto', min_samples_leaf=14,
                       min_samples_split=14, n_estimators=141, random_state=10)
rf_clf.fit(x_train,y_train)

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=12, max_features='auto', min_samples_leaf=14,
                       min_samples_split=14, n_estimators=141, random_state=10)

In [44]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Test_hyper = accuracy_score(y_test,y_pred)
print("Accuracy score", acc_score_RF_Test_hyper)

clf_report = classification_report(y_test,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[31  1  0]
 [ 0 31  1]
 [ 0  0 32]]
Accuracy score 0.9791666666666666
Classification report               precision    recall  f1-score   support

           0       1.00      0.97      0.98        32
           1       0.97      0.97      0.97        32
           2       0.97      1.00      0.98        32

    accuracy                           0.98        96
   macro avg       0.98      0.98      0.98        96
weighted avg       0.98      0.98      0.98        96



In [45]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Train_hyper = accuracy_score(y_train,y_pred)
print("Accuracy score", acc_score_RF_Train_hyper)

clf_report = classification_report(y_train,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[39  0  0]
 [ 2 36  1]
 [ 0  0 39]]
Accuracy score 0.9743589743589743
Classification report               precision    recall  f1-score   support

           0       0.95      1.00      0.97        39
           1       1.00      0.92      0.96        39
           2       0.97      1.00      0.99        39

    accuracy                           0.97       117
   macro avg       0.98      0.97      0.97       117
weighted avg       0.98      0.97      0.97       117



In [47]:
df_wine = pd.DataFrame(columns=['train_acc','Test_acc'],index=['Logistic_wine','KNN_wine','DT_wine','RF_wine'])
df_wine

,train_acc,Test_acc
Logistic_wine,NaN,NaN
KNN_wine,NaN,NaN
DT_wine,NaN,NaN
RF_wine,NaN,NaN


In [48]:
df_wine['train_acc'] = [acc_score__Logistic_Train,acc_score_KNN_Train,acc_score_DT_Train,acc_score_RF_Train_hyper]
df_wine['Test_acc'] = [acc_score_Logistic_Test,acc_score_KNN_Test,acc_score_DT_Test,acc_score_RF_Test_hyper]

In [49]:
df_wine.to_csv('df_wine.csv')
df_wine

,train_acc,Test_acc
Logistic_wine,1.000000,0.989583
KNN_wine,0.948718,0.979167
DT_wine,0.965812,0.895833
RF_wine,0.974359,0.979167
